In [1]:
''' This is a sample for celestial navigation for a stationary observer 
    © August Linnman, 2024, email: august@linnman.net
    MIT License (see LICENSE file)

    Jupyter notebook for navigation on a moving ship/vehicle. 
'''

# Importing Python libraries
from time import time
from starfix import Sight, SightTrip, get_representation, get_google_map_string, LatLon,\
                    IntersectError

In [2]:
# Estimated (DR) position at start
s1LatLon = LatLon (59, 18)
# Estimated course and speed
C_COURSE = 175
SPEED = 20

#This is the star fix for s1, the starting point

s1 = Sight (  object_name          = "Sun", \
              set_time             = "2024-06-20 06:14:38+00:00", \
              gha_time_0           = "269:35.2", \
              gha_time_1           = "284:35.1", \
              decl_time_0          = "23:26.2", \
              measured_alt         = "30:51:27.1" \
              )

In [3]:
# Star fix for ending point

s2 = Sight (  object_name          = "Sun", \
              set_time             = "2024-06-20 07:14:38+00:00", \
              gha_time_0           = "284:35.1", \
              gha_time_1           = "299:35.0", \
              decl_time_0          = "23:26.2", \
              measured_alt         = "38:34:21.6" \
              )

In [4]:
# Sight reduction

st = SightTrip (sight_start               = s1,\
                 sight_end                = s2,\
                 estimated_starting_point = s1LatLon,\
                 course_degrees           = C_COURSE,\
                 speed_knots              = SPEED)
try:
    intersections, fitness, diag_output = st.get_intersections ()
    print ("MD = " + st.get_map_developers_string ())
    print ("Starting point = " + str(get_representation(intersections[1],1)))
    print ("End point = " + str(get_representation(intersections[0],1)))
except IntersectError as ve:
    print ("Cannot perform a sight reduction. Bad sight data.\n" + str(ve))
    print ("Check the circles! " + st.get_map_developers_string())    

MD = https://www.mapdevelopers.com/draw-circle-tool.php?circles=%5B%5B6581268%2C23.4367%2C86.7554%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B5722376%2C23.4367%2C71.7571%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%5D
Starting point = (N 58°,46.1′;E 18°,0.1′)
End point = (N 58°,26.2′;E 18°,3.5′)
